In [1]:
import pandas as pd
import csv 
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
import math
import numpy as np
from scipy import spatial
import heapq
from geopy import distance
import folium

ps=PorterStemmer()

First of all we read the dataframe:

In [2]:
df = pd.read_csv("Airbnb_Texas_Rentals.csv", encoding = 'latin-1')
#df=df.drop('Unnamed: 0', axis=1)


In [3]:
df.head()

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,1,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
1,2,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW â...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...
2,3,$59,1,Houston,January 2017,'River house on island close to the city' \nA ...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...
3,4,$60,1,Bryan,February 2016,Private bedroom in a cute little home situated...,30.637304,-96.337846,Private Room Close to Campus,https://www.airbnb.com/rooms/11839729?location...
4,5,$75,2,Fort Worth,February 2017,Welcome to our original 1920's home. We recent...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location...


Now we initialize something we need

In [4]:
s_words=set(stopwords.words('english'))
#eng_word=set(nltk.corpus.words.words())
#create the list of punctuation
list_of_punct=[]
for ele in string.punctuation:
    list_of_punct.append(str(ele))
    
d={}
ind_dict=0
d_inv={}

## Step 2. : Create documents

For each row of the dataframe we create an .tsv file. We get 18259 files.

In [5]:
for idx in range(len(df)):
    l=list(df.iloc[idx])
    with open('/Users/Manfro/Data science/ADM/Homework3/Documents/doc_'+str(idx+1)+'.tsv', 'w', newline='', encoding='utf-8') as f_output:
        tsv_output = csv.writer(f_output, delimiter='\t')
        tsv_output.writerow(l)

## Step 3. : Search Engine

### Step 3.1 : Conjunctive query

After we removed all the unuseful words, like stopword and punctuation and for each word we pick the stemming. At this point, we assign to each word a label, that is a integer.

In [6]:
for i in range(len(df)):
    file1=open("/Users/Manfro/Data science/ADM/Homework3/Documents/doc_"+str(i+1)+".tsv",encoding=("utf-8"))
    line=file1.read()
    words=line.split('\t')
    s=words[5]+' '+words[8]
    s=s.replace("\\n",' ')
    tokens = nltk.word_tokenize(s)
    filtered_sentence=[word for word  in tokens if not word in list_of_punct ]
    filtered_sentence=[word for word  in filtered_sentence if not word in s_words]
    s=filtered_sentence

    for w in s:
        if ps.stem(w) not in d:
            d[ps.stem(w)]=ind_dict
            ind_dict+=1
        if d[ps.stem(w)] not in d_inv:
            d_inv[d[ps.stem(w)]]=['doc_'+str(i+1)]
        else:
            d_inv[d[ps.stem(w)]].append('doc_'+str(i+1))

Now we create the inverted dictionary, that has as keys the labels of the first dictionary and for values a set of the names of documents that contains the word.

It's time to insert the query! 

In [7]:
query=input('Insert the query: ').split()

Insert the query: grocery beach


Non we implement the search engine. It's a function that given the query, returns a list of sorted index of documents that contains all the "important" words of the query. First of all, we splitted the string so we have a list of strings. We pick only the words we think that are useful, and we check in what document they are. We get a list of sets. Then we apply the operator intersection that gives us all the documents that contains all the words of the query. Since we're interessend only to the index of documents, using the method startswith and the method replace we can access to the index of documents, that we convert in *int* format and then we sort this list. A the end the function returns this list.

In [8]:
def search_engine(query):
    setList=[]
    for word in query:
        if word not in s_words:
            setList.append(set(d_inv[d[ps.stem(word)]]))
    s=set.intersection(*setList)
    
    l=[]
    for doc in s:
        if doc.startswith('doc_')==True:
            l.append(doc)
            
    l_int=[]
    for item in l:
        l_int.append(int(item.replace('doc_',''))-1)
    
    l_int_sort=sorted(l_int)
    return(l_int_sort)

We oserved that the number in the document name is the index-1 of the rows of the dataframe, so we create a list of index and for each index we copy the rows in a new dataframe(df_)

In [9]:
l_int_sort=search_engine(query)
df.iloc[l_int_sort,[8,5,3,9]]

,title,description,city,url
1874,Nature's Abundance,"My place is close to the Moody Gardens, beach,...",Galveston,https://www.airbnb.com/rooms/13802467?location...
1892,Beachfront Paradise Galveston Condo,"My place is good for couples, solo adventurers...",Galveston,https://www.airbnb.com/rooms/13307473?location...
2251,Maggie's Imperfect Perfection,"The Gulf of Mexico is beautiful, and this firs...",Port Aransas,https://www.airbnb.com/rooms/16854214?location...
2262,1 BDRM CONDO WALKING DIST TO BEACH,"Condo has wifi, cable, swimming pool, picnic a...",Port Aransas,https://www.airbnb.com/rooms/893772?location=C...
2652,Cozy 1brm furnished condo,Our condo is fully furnished with all your nee...,Corpus Christi,https://www.airbnb.com/rooms/11486854?location...
3214,Laguna Tiki Bar House,The perfect family getaway for all coastal lov...,Corpus Christi,https://www.airbnb.com/rooms/18018388?location...
5746,Island Hideaway,Fully equipped kitchenette suite in 1/2 of hou...,Port Aransas,https://www.airbnb.com/rooms/17193877?location...
8969,Siesta by the Sea Shore with a private pool,My place is close to family-friendly activitie...,South Padre Island,https://www.airbnb.com/rooms/14024642?location...
9672,Ocean is the Backyard!,In the center of town: \nWithin Walking Distan...,South Padre Island,https://www.airbnb.com/rooms/11588454?location...
10347,Private Room in The Austin Oasis House Lake Tr...,Welcome to the Lake Travis area! You are going...,Austin,https://www.airbnb.com/rooms/15903071?location...


### 3.2: Conjunctive query & Ranking score

In this chunk we created the second inverted dictionary, as written in the exercise.

In [10]:
d_inv2={}
N=len(df)
for label in d_inv.keys():
    d_inv2[label]=[]
    den=len(set(d_inv[label]))
    s_to_ex=[]
    for item in d_inv[label]:
        if item not in s_to_ex:
            index=int(item.replace('doc_',''))-1
            s=str(df.iloc[index,5])+' '+str(df.iloc[index,8])
            s=s.replace("\\n",' ')
            doc_len=len(s.split())
            count=d_inv[label].count(item)
            d_inv2[label].append((item,(count/doc_len)*math.log10(N/len(set(d_inv[label])))))
            s_to_ex.append(item)

In [11]:
query=input('Insert the query: ').split()
k=int(input('Insert the number of documents you want to print (k): '))

Insert the query: grocery beach
Insert the number of documents you want to print (k): 7


How we did in the previous step, we find the indexes of documents that contains all the words of the query using the intersection among sets. We use the function that we have written previuosly

In [12]:
l_int_sort=search_engine(query)

Now we compute the tfidf score for the query. 

In [13]:
y=[]
N1=len(df)+1
for word in query:
    if word not in s_words:
        n=query.count(word)
        den=len(query)
        den1=len(set(d_inv[d[ps.stem(word)]]))
        y.append((n/den)*math.log10(N1/den1))
        
y=np.asarray(y)
norm=np.linalg.norm(y)
if norm!=0:
    y=y/norm

For compute the cosine similarity between the query and each documents, we have to create a vector for each documents. For each document, the vector is made by the tfidf score of the words associated to each word of the query.

In [14]:
points=[]
for i in range(len(l_int_sort)):
    x=[]
    name_doc='doc_'+str(l_int_sort[i]+1)
    for w in query:
        if w not in s_words:
            sw=ps.stem(w)
            if sw in d:
                l=d_inv2[d[sw]]
                for tup in l:
                    if tup[0]==name_doc:
                        x.append(tup[1])
            else:
                x.append(0)
    points.append(x)

Now we have a list of list (points) that contains all the vectors we introduce previously. Now we compute the cosine similarity between the query and each vector and we store it in a tuple made by (cosine_similarity_doc_i, i). We store this tuples in a heap: each knots is a tuple and the key is the cosine similarity.

In [15]:
heap=[]

for x,i in zip(points,range(len(l_int_sort))):
    x=np.asarray(x)
    norm=np.linalg.norm(x)
    if norm!=0:
        x=x/norm
    Sim=((1-spatial.distance.cosine(x,y),l_int_sort[i]))
    heapq.heappush(heap,Sim)

Now we store each k-tuples in a list, ordered by the value of cosine similarity.

In [16]:
l=[]
max_heap=heapq.nlargest(k,heap)
for tup in max_heap:
    l.append(float(str(tup[0])[0:5]))

We create a new dataframe that contains all the row we need

In [17]:
df_1=pd.DataFrame()

for tup in max_heap:
    idx=tup[1] 
    row=pd.DataFrame()
    row=df.iloc[[idx]]
    df_1=pd.concat([df_1,row])
df_1=df_1.filter(['title','description','city','url'])
df_1['Similarity']=l
df_1

,title,description,city,url,Similarity
18194,Laguna Tiki Bar House,The perfect family getaway for all coastal lov...,Corpus Christi,https://www.airbnb.com/rooms/18018388?location...,0.999
18089,Island Hideaway,Fully equipped kitchenette suite in 1/2 of hou...,Port Aransas,https://www.airbnb.com/rooms/17193877?location...,0.999
13520,Laguna Tiki Bar House,The perfect family getaway for all coastal lov...,Corpus Christi,https://www.airbnb.com/rooms/18018388?location...,0.999
11844,3BR Great Location Peaceful and Centric Apartment,"Our 3 bedroom family condo is very spacious, t...",South Padre Island,https://www.airbnb.com/rooms/15927683?location...,0.999
11547,Pueblos del Padre,This beautiful new condo is a ground floor 2 b...,South Padre Island,https://www.airbnb.com/rooms/19020727?location...,0.999
10347,Private Room in The Austin Oasis House Lake Tr...,Welcome to the Lake Travis area! You are going...,Austin,https://www.airbnb.com/rooms/15903071?location...,0.999
5746,Island Hideaway,Fully equipped kitchenette suite in 1/2 of hou...,Port Aransas,https://www.airbnb.com/rooms/17193877?location...,0.999


# Step 4: Define your score


We used the query insered before.

Now we crate a new dictionary that has a keys, the documents, and as value our persona tfidf score. First of all we initialize the values of the dictionary as an empty list.

In [57]:
my_score_dic={}
for idx in l_int_sort:
    my_score_dic[idx]=[]

In [58]:
df_2=pd.DataFrame()

for idx in l_int_sort:
    row=pd.DataFrame()
    row=df.iloc[[idx]]
    df_2=pd.concat([df_2,row])

Our first idea is to create a gradutory for the city. More precisely, after we get the documents that contais all the words od the query we want to create a graduatory based on preferences of the user. So the program will print all the cities with a personal code (an integer) and the user will insert a list of number that represent his/her preferences. 

For example, if we find this cities: (Nacogdoches', 'Amarillo', 'Austin'), the programm associates this code:
    Nacogdoches: 0 
    Amarillo: 1 
    Austin:2
    
If the user insert this list of numbers: [2,0,1], he's saying that the city that he/she would like to live is Austin, then Nacogdoches and so on ans so for.

So when we find this graduatory, we will assign to each city a personal point. We were inspired by the graduatory of a Formula1 race: the first pick 25 points, the second 18 points and so on and so for. For assign the point we used the Fibonacci's sequences, in this way we think and hope to give a good distribution for the points, we should have the biggest gap between first and second, a smaller than the previously gap between second and third and so on. In this way we hope that we are giving the right priority to the cities where the users would like to live. Let's have a look to the code: first of all we used a set for find the cities. 

In [33]:
cities=list(set(df_2['city']))

In [34]:
city_d={}
i=0
for city in cities:
    city_d[city]=i
    i+=1

In [35]:
city_d_inv={}
i=0
for city in cities:
    city_d_inv[i]=[city]
    i+=1

So we have this dictionaries

In [36]:
city_d

{'Corpus Christi': 0,
 'Matagorda': 1,
 'Port Aransas': 2,
 'Austin': 3,
 'South Padre Island': 4,
 'Galveston': 5}

In [37]:
city_d_inv

{0: ['Corpus Christi'],
 1: ['Matagorda'],
 2: ['Port Aransas'],
 3: ['Austin'],
 4: ['South Padre Island'],
 5: ['Galveston']}

Now the user has to insert his preferences, so a list of number where each number is associated to a city

In [38]:
preferences=list(map(int, input('Please insert the ranking of preferences: ').split()))

Please insert the ranking of preferences: 4 3 2 5 1 0 


Now we are computing the scores for each city using Fibonacci's sequence

In [39]:
k=[1,1]
i=1
while len(k)<=len(cities)+1:
    k.append(k[i]+k[i-1])
    i+=1
k=sorted(k[2:],reverse=True)

And we assign the score to each city

In [40]:
idx=0
for i in preferences:
    city_d_inv[i].append(k[idx])
    idx+=1

In [41]:
city_d_inv

{0: ['Corpus Christi', 2],
 1: ['Matagorda', 3],
 2: ['Port Aransas', 8],
 3: ['Austin', 13],
 4: ['South Padre Island', 21],
 5: ['Galveston', 5]}

In [42]:
for idx in l_int_sort:
    city=df_2.loc[idx]['city']
    
    my_score_dic[idx]=city_d_inv[city_d[city]][1]

Now each city has its score and the minimum score is 2. Now since we can have different houses in each city, we have also to classificate it for the price. Let's insert the maximum price that user wants to pay. After than we decrase the score of each documents this quantity: (price_doc_i-insered_price)/max_price, where *price_doc_i* is the price of the location associated to the document *i*, *insered_price* is the price isered by the user and *max_price* is the maximum price among the prices we find as result of the query. Note that, if the price of a location is smaller than the price insered, we're increasing the score.

In [43]:
prices=list(df_2['average_rate_per_night'])

In [44]:
cast_p=[]
for p in prices:
    p=p.replace('$',' ')
    cast_p.append(int(p))

In [45]:
max_price=max(cast_p)

In [46]:
price=int(input('Please insert the price you would like to pay: '))

Please insert the price you would like to pay: 100


In [47]:
keys=list(my_score_dic.keys())
for i in range(len(keys)):
    my_score_dic[keys[i]]-= (cast_p[i]-price)/max_price

Now we store all the documents with their score in a heap, than we will pick the first *k* values

In [48]:
heap_s=[]
for key in keys:
    t=(my_score_dic[key],key)
    heapq.heappush(heap_s,t)

In [49]:
nscore=int(input('Please insert the number of locations you want to see:'))

Please insert the number of locations you want to see:7


In [50]:
max_heap_s=heapq.nlargest(nscore,heap_s)
max_heap_s

[(20.941903584672435, 11844),
 (20.894932014833127, 11547),
 (20.876390605686034, 11186),
 (20.558714462299136, 9672),
 (20.123609394313966, 8969),
 (13.058096415327565, 10347),
 (8.012360939431396, 18089)]

In [51]:
df_3=pd.DataFrame()

for tup in max_heap_s:
    idx=tup[1]
    row=df_2.loc[[idx],]
    df_3=pd.concat([df_3,row])
    
df_3=df_3.filter(['title','description','city','url'], axis=1)
df_3=df_3.drop_duplicates(subset=['description','title'])
df_3=df_3.reset_index(drop=True)
r=list(range(1,len(df_3)+1))
df_3.insert(0,'Ranking',r)
df_3

,Ranking,title,description,city,url
0,1,3BR Great Location Peaceful and Centric Apartment,"Our 3 bedroom family condo is very spacious, t...",South Padre Island,https://www.airbnb.com/rooms/15927683?location...
1,2,Pueblos del Padre,This beautiful new condo is a ground floor 2 b...,South Padre Island,https://www.airbnb.com/rooms/19020727?location...
2,3,Marlin Star Beachside Paradise,"Short walk (3-4 minute) to the beach, SPI beac...",South Padre Island,https://www.airbnb.com/rooms/18627929?location...
3,4,Ocean is the Backyard!,In the center of town: \nWithin Walking Distan...,South Padre Island,https://www.airbnb.com/rooms/11588454?location...
4,5,Siesta by the Sea Shore with a private pool,My place is close to family-friendly activitie...,South Padre Island,https://www.airbnb.com/rooms/14024642?location...
5,6,Private Room in The Austin Oasis House Lake Tr...,Welcome to the Lake Travis area! You are going...,Austin,https://www.airbnb.com/rooms/15903071?location...
6,7,Island Hideaway,Fully equipped kitchenette suite in 1/2 of hou...,Port Aransas,https://www.airbnb.com/rooms/17193877?location...


# Bonus Step: Make a nice visualization!



In this part, we create our map. First of all, will be required to insert the coordinates, latitude and longitude as a pair, of th centre of the circle and then will be request to insert the radius of the circle. 

In [52]:
location=list(map(float, input('Please enter the Latitude and the Longitude: ').split()))
rad=int(input('Please enter the radius (m):'))

Please enter the Latitude and the Longitude: 32.892048 -96.871741
Please enter the radius (m):2000


In [53]:
my_values1 = df.filter(["latitude","longitude",'average_rate_per_night','url','title'])

Now we removed all duplicates and rows that have a *na*. We converted the dataframe in a list.

In [54]:
my_values = my_values1.dropna(how = "any")
my_values= my_values.drop_duplicates()

In [55]:
locations = my_values[['latitude', 'longitude','average_rate_per_night','url','title']]
locationlist = locations.values.tolist()

Finally we created the map. First of all, we choosed as tiles *Stamen Terrain* and placed a blue marker and as icon a flag, for indicate the position that the user has insered.
Than we added to the map a marker, each time the house is inside the circle. The markers have different colours: let *p* the average rate per night
    
    - *lightblue* if p<=30
    - *green*     if 30<p<=50
    - *orange*    if 50<p<=70
    - *red*       if 70<p<=90
    - *darkred*   if 90<p<=140
    - *black*     if p>140
    
So they are divided by the price.

In [56]:
map2 = folium.Map(location=location,tiles='Stamen Terrain', zoom_start=12)
folium.Marker(location, icon=folium.Icon(color='blue', icon='flag')).add_to(map2)
folium.Circle(location=location, radius=rad, color='#3186cc', fill=True, fill_color='#3186cc').add_to(map2)

for l in locationlist:
    point=[l[0],l[1]]
    url=l[3]
    title=l[4]
    if distance.distance(location, point).m<rad:
        p=int(l[2].replace('$',' '))
        if p<=30:
            color='lightblue'
        if 30<p<=50:
            color='green'
        if 50<p<=70:
            color='orange'
        if 70<p<=90:
            color='red'
        if 90<p<=140:
            color='darkred'
        if p>=140:
            color='black'
        folium.Marker(point, icon=folium.Icon(color=color, icon='home'),tooltip='<b>'+title+'<b>',
                      popup = folium.Popup("<a href=" + url + ">" + url + " </a>")).add_to(map2)
        
map2.save('Map.html')
